In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from pytexit import py2tex
from IPython.display import Markdown
from gekko import GEKKO
import pandas as pd
from matplotlib import rcParams
import os
import sys
from copy import copy


%matplotlib widget
#%matplotlib inline

# Look for custom python codes in the Radigan Engineering Vault...
sys.path.insert(1, 'C:\\\\RadiganVault\\\\Python')

import RadiganEngineeringDefaults as RE
from RadiganEngineering_Materials_Library import *
from RadiganEngineeringTorsionSpring import *
from RadiganEngineeringBellevilleSpring import *

# Use higher quality figures in PDF exports
#rcParams['figure.dpi']= 300
#rcParams['figure.figsize'] = [8, 6]
set_matplotlib_formats('png', 'pdf')

In [4]:
Module3 = GEKKO(remote=False)    # create GEKKO model for dx/dt
#Module3 = GEKKO()    # create GEKKO model for dx/dt


#DTS_TC114.REQ_Flat_Load = 15000 * ureg.lbf # Preload required is 15 kip
#DTS_TC114.REQ_Travel    = 3.0   * ureg.mm  # Require at least 3mm of Travel
#DTS_TC114.Calculate_Spring_Stack()


t_sim = 10 * ureg.sec # Number of seconds to simulate
points_per_second = 10

Module3.time = np.linspace(0, t_sim.magnitude, (t_sim.magnitude * points_per_second) + 1) * ureg.sec # time points

print(Module3.time)
# dt is in the units of ________.

rho       = Module3.Param(1000.0)  # water density (kg/m^3)
Area      = Module3.Param(1.0)  # tank area (m^2)
Valve_Constant = Module3.Param(50.0) # Valve Constant

valve = np.zeros(len(Module3.time)) # u = valve % open
valve[21:70] = 100.0  # open valve between 2 and 7 seconds
#print(valve)
#print(len(valve))
#print(Module3.time)
#print(len(Module3.time))
Valve_State = Module3.Param(valve)

Level     = Module3.Var(0) # Start with an empty tank

# calculate derivative of the Level
Module3.Equation(Level.dt()  == (Valve_Constant/(rho*Area)) * Valve_State)

# solve ODEs and plot
# https://gekko.readthedocs.io/en/latest/imode.html
Module3.options.IMODE = 4 # dynamic simulation
#Module3.options.IMODE = 3 # Steady State simulation

# Type: Integer, Input/Output
# Default Value: 0
# Description: Cold start model: 0=warm start, 1=cold start, 2=decompose problem
#Module3.options.COLDSTART = 2

#Module3.options.SOLVER = 1
#Module3.options.MAX_ITER = 1000

#---------------------
#Module3.options.NODES = 6
#---------------------

#Module3.options.TIME_SHIFT=0

Module3.solve(disp=False)

print(Module3.time)

print('Starting Level = {0:.3f}, Final Level = {1:.3f}'.format(Level.value[0], Level.value[-1]))

plt.figure()
plt.subplot(2,1,1)
plt.plot(Module3.time,Level, marker='',linewidth=1,label='Tank Level')

plt.grid(True)
#plt.xlabel('Time')
plt.ylabel('Tank Height')
plt.legend()
plt.show()

plt.subplot(2,1,2)
plt.plot(Module3.time,Valve_State, color = 'red', linestyle = '--', marker='',linewidth=1,label='Valve State')

plt.grid(True)
#plt.xlabel('Time (seconds)')
plt.ylabel('Valve % Open State')
plt.legend()
plt.show()

plt.suptitle('Tank Level Simulation')



[0.0 0.1 0.2 0.30000000000000004 0.4 0.5 0.6000000000000001 0.7000000000000001 0.8 0.9 1.0 1.1 1.2000000000000002 1.3 1.4000000000000001 1.5 1.6 1.7000000000000002 1.8 1.9000000000000001 2.0 2.1 2.2 2.3000000000000003 2.4000000000000004 2.5 2.6 2.7 2.8000000000000003 2.9000000000000004 3.0 3.1 3.2 3.3000000000000003 3.4000000000000004 3.5 3.6 3.7 3.8000000000000003 3.9000000000000004 4.0 4.1000000000000005 4.2 4.3 4.4 4.5 4.6000000000000005 4.7 4.800000000000001 4.9 5.0 5.1000000000000005 5.2 5.300000000000001 5.4 5.5 5.6000000000000005 5.7 5.800000000000001 5.9 6.0 6.1000000000000005 6.2 6.300000000000001 6.4 6.5 6.6000000000000005 6.7 6.800000000000001 6.9 7.0 7.1000000000000005 7.2 7.300000000000001 7.4 7.5 7.6000000000000005 7.7 7.800000000000001 7.9 8.0 8.1 8.200000000000001 8.3 8.4 8.5 8.6 8.700000000000001 8.8 8.9 9.0 9.1 9.200000000000001 9.3 9.4 9.5 9.600000000000001 9.700000000000001 9.8 9.9 10.0] second
[0.0 0.1 0.2 0.30000000000000004 0.4 0.5 0.6000000000000001 0.7000000000

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Tank Level Simulation')